<a href="https://colab.research.google.com/github/SaurabhSRP/POC-CNN/blob/main/Image_net_classification_for_CNN_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Image net classification for CNN - transfer learning**

https://keras.io/api/applications/

we will create a python template such that when applying we will take the library and apply it to our generic template

In [ ]:
pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from keras.layers import Input,Lambda,Dense,Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np

from glob import glob
import matplotlib.pyplot as plt

In [ ]:
IMAGE_SIZE=[224,224] #resize all images as VGG16 for made for images of 224x244

In [ ]:
train_path=""
valid_path=""

In [ ]:
vgg=VGG16(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)
#add preprocessing layer to the front of VGG
#image_size[3], here 3 refers to 3 RGB channel , for grey scale its [1]
#include_top=False , removes the last layer as we dont have 1000 different categories of data
#because Imagenet competition has 1000 different category of data

In [ ]:
#dont train existing weights
for layer in vgg.layers:
  layer.trainable=False    #we dont have to train the vgg16 layers , its already trained ,weights are fixed , and VGG is pretrained so we are using it 



In [ ]:
folders=glob('datasets/train/*') #we can check all folders and no. of classification to be done

In [ ]:
x=flatten()(vgg.output) #flatten the last layer of VGG

In [ ]:
prediction=Dense(len(folders),activation='softmax')(x)   #this will append all my categories into the last layer

In [ ]:
#create a model object
model=Model(inputs=vgg.input,output=prediction)

In [ ]:
model.summary()

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('Datasets/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('Datasets/Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

In [ ]:
#fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

In [ ]:
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')


In [ ]:
# accuracies
plt.plot(r.history['acc'], label='train acc')
plt.plot(r.history['val_acc'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
import tensorflow as tf

from keras.models import load_model

model.save('facefeatures_new_model.h5')

##Just remove vgg16 for other transfer learning alogrithms mentioned in the keras list and use this template